# 00 · Colab Bootstrap
Clone repo, install deps, and load data (Parquet or read-only DB).

In [ ]:
# --- Colab bootstrap: clone repo, install deps, choose data source ---
import os, subprocess, sys, getpass, glob

USER   = "YOUR_USER"
REPO   = "YOUR_REPO"
BRANCH = "main"

# Clone shallow
if not os.path.exists(REPO):
    !git clone --depth 1 https://github.com/{USER}/{REPO}.git -b {BRANCH}
%cd {REPO}

# Install
%pip -q install -U pip wheel setuptools
%pip -q install -r requirements.txt
%pip -q install polars duckdb scikit-learn lightgbm xgboost plotly jupyterlab_widgets

# Data source: Parquet or read-only DB
USE_DB = False  # flip to True if you have a read-only DATABASE_URL
if USE_DB:
    DATABASE_URL = os.getenv("DATABASE_URL") or getpass.getpass("DATABASE_URL (read-only): ")
    import pandas as pd
    import sqlalchemy as sa
    engine = sa.create_engine(DATABASE_URL)
    q = "select * from matches where season >= 2022 order by date;"
    df = pd.read_sql(q, engine)
else:
    import pandas as pd
    files = sorted(glob.glob("data/exports/**/*.parquet", recursive=True))
    if not files:
        print("No local parquet found. Try running the scraper or switch USE_DB=True.")
        df = pd.DataFrame()
    else:
        df = pd.concat((pd.read_parquet(p) for p in files), ignore_index=True)

df.shape


In [ ]:
df.head(3)


In [ ]:
# (Optional) Save executed notebook as HTML in reports/
!jupyter nbconvert --to html --output-dir=reports --no-input notebooks/00_colab_bootstrap.ipynb
